In [1]:
import facebook
import os
import requests
import pandas as pd

# Leggo Configuration File
config = {}
config_path = os.path.join(os.environ['HOME'], 'opendata')
config_name = 'config.py'
config_file = os.path.join(config_path,config_name)
exec(open(config_file).read(),config)
token=config['TOKEN_FB']

# Utilizzo token generoato manualmente con la mia utenza sul Tool di Esplorazioni API Graph
graph = facebook.GraphAPI(access_token=token, version='2.7')

# Directory di Staging
dir_df = os.path.join(os.path.abspath(''),'stg')

In [16]:
# Recupero tutti i posts di Salvini (ID e Data Creazione)
df_filename = r'df_posts.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df_posts = pd.read_pickle(df_fullpath)

In [3]:
df_posts.head(5)

,ID,Post_Date
0,252306033154_10151694974523155,2013-09-26T17:54:20+0000
1,252306033154_10152854711393155,2015-03-29T10:07:10+0000
2,252306033154_10151493392598155,2013-06-13T08:15:41+0000
3,252306033154_10151156433468155,2012-12-11T21:57:01+0000
4,252306033154_10152607210623155,2014-12-21T12:42:02+0000


In [30]:
#Esempio di get_likes
#graph.get_connections(id='252306033154_10151694974523155', connection_name='likes', summary='true')['summary']['total_count']

1049

In [36]:
def get_likes(post):
    likes = None
    try:
        likes = graph.get_connections(id=post['ID'], connection_name='likes', summary='true')['summary']['total_count']
    except:
        likes = None
    return likes

In [39]:
# Molto Lento, impiega ca. 2 ore su 16000 post, va ottimizzato (thread, parallel, etc)
df_posts['Likes'] = df_posts.apply(lambda x: get_likes(x), axis=1)

In [40]:
df_posts.head(5)

,ID,Post_Date,Likes
0,252306033154_10151694974523155,2013-09-26T17:54:20+0000,1049.0
1,252306033154_10152854711393155,2015-03-29T10:07:10+0000,8093.0
2,252306033154_10151493392598155,2013-06-13T08:15:41+0000,251.0
3,252306033154_10151156433468155,2012-12-11T21:57:01+0000,110.0
4,252306033154_10152607210623155,2014-12-21T12:42:02+0000,5031.0


In [41]:
# Salvo il DF per non ripetere la Chiamata
df_filename = r'df_posts_likes.pkl'
df_fullpath = os.path.join(dir_df, df_filename)
df_posts.to_pickle(df_fullpath)